### Active Learning for sample selection

Active Learning can also be used to create a diverse and representative selection of the data, by using representation-based query strategies, without need of the labels and of a prediction/classification model. This can be very useful if our dataset is imbalanced, or if some outliers/extreme points are very significant to our analysis and we need to be sure to include them even if they are a minority, as random sampling is likely to overlook them.
For this example we are going to use a subset of [_MNIST_](https://archive.ics.uci.edu/dataset/683/mnist+database+of+handwritten+digits) dataset, loaded from _sklearn_.

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, models, transforms

from activelearning.AL_selection import selection_AL
from activelearning.queries.representative.coreset_query import query_coreset

np.random.seed(123)

In [ ]:
# convert images to tensors of acceptable sizes for ResNet and normalize
preprocess = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# import data
train_full = datasets.MNIST(root="./data", train=True, download=True, transform=preprocess)
test_full = datasets.MNIST(root="./data", train=False, download=True, transform=preprocess)

# we are going to use a subset of 1500 images for this example
y_train = train_full.targets[:1000,]
y_test = test_full.targets[:500,]

train = Subset(train_full, indices=range(1000))
test = Subset(test_full, indices=range(500))

# to use batches instead of one at a time
loaded_train = DataLoader(train, batch_size=64, shuffle=False)
loaded_test = DataLoader(test, batch_size=64, shuffle=False)

_MNIST_ consists of images of handwritten digits from 0 to 9. When working with image data, we need to first extract features using for example a pretrained model. In this case, we will use [ResNet18](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html) for feature extraction.

In [ ]:
# import ResNet model
resnet = models.resnet18(pretrained=True)

# remove classification layer
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])
# set to evaluation mode
resnet.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet.to(torch.device(device))

train_features_list = []
with torch.no_grad():
    for images, _ in loaded_train:
        features_by_resnet = resnet(images.to(device))
        train_features_list.append(features_by_resnet)

test_features_list = []
with torch.no_grad():
    for images, _ in loaded_test:
        features_by_resnet = resnet(images.to(device))
        test_features_list.append(features_by_resnet)


train_features = torch.cat(train_features_list, dim=0).squeeze()

train_features = train_features.cpu().numpy()

test_features = torch.cat(test_features_list, dim=0).squeeze()
test_features = test_features.cpu().numpy()

To compare how different query strategies perform, we can use the *strategy_comparison* function and pass the strategies to be used. We can also pass more than one number of instances, to check whether a different batch size influences performance. *plot_results* can be used to immediatly plot the output from *strategy_comparison*, or a custom graph can be created from the scores data frame.

In [ ]:
selected_data, selected_idxs = selection_AL(
    X_train=None, X_pool=train_features, query_strategy=query_coreset, n_instances=1, n_queries=20
)

print(pd.Series(y_train[selected_idxs]).value_counts())